# Self-Supervised Monocular Depth Estimation

In [9]:
from dataloader import KittiDataset
from transformation import *
from model import Model

import torch
from torch.utils.data import DataLoader

import os
import random
import torchvision.transforms as transforms
import torchvision.transforms.functional as tF

## Data Loading

In [15]:
train_path = os.path.join(os.getcwd(), "data/train")
test_path = os.path.join(os.getcwd(), 'data/test')

In [21]:
train_transform = transforms.Compose([
    ToResizeImage(),
    ToRandomFlip(),
    ToTensor(),
    AugumentImagePair(),
])
test_transform = transforms.Compose([
    ToResizeImage(),
    ToTensor(),
])

In [22]:
train_set = KittiDataset(train_path, 'train', transform = train_transform)
test_set = KittiDataset(test_path, 'test', transform = test_transform)

train_loader = DataLoader(train_set, batch_size = 5, shuffle = True)
test_loader = DataLoader(test_set, batch_size = 1, shuffle = False)

# for batch_step, batch_sample in enumerate(train_loader):
#     print(batch_step, batch_sample['left_image'].shape)
#     break  

## Implement CNN using PyTorch

In [23]:
model = Model(device='cpu', 
              epochs=1, 
              save_per_epoch='none', 
              img_height=256, 
              img_width=512, 
              model_path='output_model', 
              disp_path='output_disp')

In [24]:
model.train(train_loader)

RuntimeError: Calculated padded input size per channel: (2 x 10). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

In [ ]:
model.test(test_loader)